In [ ]:
import os
import json
from json import JSONDecodeError
import numpy as np
from tqdm import tqdm
import torch
import gc

from collections import defaultdict

from util.constants import Topic
from util.helpers import cos_sim

In [ ]:
# Models
from sentence_transformers import SentenceTransformer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens').to(device)

def model(titles):
    return sbert_model.encode(titles)

Get latents for all videos

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
RESULTS_DIR = os.path.join("..", "data", "title-latents")

video_results_dir = os.path.join("..","..","DATA","title-latents","videos")
channel_results_dir = os.path.join(RESULTS_DIR, "channels")

if not os.path.exists(video_results_dir):
    os.makedirs(video_results_dir)

def get_done_list(dir):
    return [nm.replace(".json",'').replace('.pt','') for nm in os.listdir(dir)]

In [ ]:
# Run the code in batches
for channel,videos in tqdm(channel_videos_dict.items()):
    done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
    ids = [vid["id"] for vid in videos[:30]]
    try:
        with open(done_path, "r") as f:
            done_ids = json.load(f)
        if set(ids).issubset(done_ids): # Every id is done
            continue
    except FileNotFoundError:
        pass

    titles = [vid["title"] for vid in videos[:30]]
    
    batch_latents = np.array(model(titles))

    path = os.path.join(video_results_dir, f"{channel}.pt")
    torch.save(batch_latents, path)
    with open(done_path, "w") as f:
        json.dump(ids, f)

    torch.cuda.empty_cache()

Channel stats

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
# Calculate channel results
for channel,videos in tqdm(channel_videos_dict.items()):
    in_path = os.path.join(video_results_dir,channel+".pt")
    result_list = torch.load(in_path)

    channel_mean = result_list.mean(axis=0)
    channel_result = {
        "std": float(result_list.std()),
        "len": len(result_list),
    }

    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    with open(filepath, "w") as f:
        json.dump(channel_result, f)
    filepath = os.path.join(RESULTS_DIR, "channels_mean", f"{channel}.pt")
    torch.save(channel_mean, filepath)

Category stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "channel2category.json"), "r") as f:
    channel2cat = json.load(f)

In [ ]:
# Make list of results per channel for each category
category_results_list = defaultdict(list)
for channel in tqdm(get_done_list(channel_results_dir)):
    cat = channel2cat[channel]
    results = {}
    for folder,ext in [("channels",".json"), ("channels_mean",".pt")]:
        filepath = os.path.join(RESULTS_DIR, folder, channel+ext)
        try:
            if ext == ".json":
                with open(filepath, "r") as f:
                    results.update(json.load(f))
            elif ext == ".pt":
                results.update({"mean": torch.load(filepath)})
        except JSONDecodeError:
            print(f"couldn't open {channel}; deleting file")
            os.remove(filepath)
    category_results_list[cat].append(results)

In [ ]:
# Calculate category results
for cat,stats_list in category_results_list.items():
    mean_list = np.array([channel_stats["mean"] for channel_stats in stats_list])
    std_list = np.array([channel_stats["std"] for channel_stats in stats_list])
    category_mean = mean_list.mean(axis=0)
    category_result = {
        "std": float(mean_list.std()),
        "avg_std_of_channels": std_list.mean(),
        "len": len(mean_list),
    }

    filepath = os.path.join(RESULTS_DIR, "categories", f"{cat}.json")
    with open(filepath, "w") as f:
        json.dump(category_result, f)
    filepath = os.path.join(RESULTS_DIR, "categories_mean", f"{cat}.pt")
    torch.save(category_mean, filepath)

Most Representative

In [ ]:
channels = get_done_list(channel_results_dir)

channel_means = []
for channel in tqdm(channels):
    path = os.path.join(RESULTS_DIR, "channels_mean", channel+".pt")
    channel_means.append(torch.load(path))
channel_means = torch.Tensor(channel_means)

In [ ]:
categories = Topic._member_names_

category_means = []
for cat in tqdm(categories):
    path = os.path.join(RESULTS_DIR, "categories_mean", cat+".pt")
    category_means.append(torch.load(path))
category_means = torch.Tensor(category_means)

all_means = torch.cat((channel_means, category_means), axis=0)

In [ ]:
def cos_sim(arrayA, arrayB):
    """[N,D], [M,D] -> [N,M]"""
    normA = torch.norm(arrayA, p=2, dim=1)[:,None]
    normB = torch.norm(arrayB, p=2, dim=1)[:,None]
    return arrayA/normA @ (arrayB/normB).T

def dist(tensorA, tensorB):
    """[N,D], [M,D] -> [N,M]"""
    return ((tensorA[:,:,None] - tensorB.transpose()[None,:,:])**2).mean(1)**.5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

all_means = all_means.to(device)
best_similarity = torch.zeros(len(all_means)).to(device)
best_repr = [""]*len(all_means)

batch_size = 128
for i in tqdm(range(len(channels)//batch_size+1)):
    done_ids = []
    done_means = torch.zeros((0,768), device=device)
    for channel in channels[i*batch_size:(i+1)*batch_size]:
        done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
        with open(done_path, "r") as f:
            done_ids.extend(json.load(f))
        if not done_ids:
            continue
        
        filepath = os.path.join(video_results_dir, channel+".pt")
        loaded_means = torch.Tensor(torch.load(filepath)).to(device)
        done_means = torch.cat((done_means, loaded_means), dim=0)

    if len(done_means) < 1:
        continue

    similarities = torch.max(cos_sim(all_means, done_means), dim=-1)
    update_best = (similarities.values > best_similarity).nonzero()
    best_similarity[update_best] = similarities.values[update_best]
    for idx in update_best:
        best_repr[idx] = done_ids[similarities.indices[idx]]

In [ ]:
vid2title = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        videos_info = json.load(f)
    vid2title.update({vid["id"]:vid["title"] for channel_vids in tqdm(videos_info.values()) for vid in channel_vids})

In [ ]:
channels_best_repr = {k:{"sim": sim.item(), "vid_id": id, "repr": vid2title[id]} for k,sim,id in zip(channels,best_similarity,best_repr) if id}
with open(os.path.join(RESULTS_DIR, "channels_best_repr.json"), "w") as f:
    json.dump(channels_best_repr, f)

In [ ]:
categories_best_repr = {k:{"sim": sim.item(), "vid_id": id, "repr": vid2title[id]} for k,sim,id in zip(categories,best_similarity[-9:],best_repr[-9:]) if id}
with open(os.path.join(RESULTS_DIR, "categories_best_repr.json"), "w") as f:
    json.dump(categories_best_repr, f)

Video latent deviation

In [ ]:
for channel in tqdm(channels):
    done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
    with open(done_path, "r") as f:
        done_ids = json.load(f)
    if not done_ids:
        continue
    
    filepath = os.path.join(video_results_dir, channel+".pt")
    loaded_means = torch.load(filepath)
    done_means = loaded_means

    if len(done_means) < 1:
        continue

    distances = dist(channel2mean[channel][None], done_means)[0]

    result = {
        "mean": float(distances.mean()),
        "datapoints": [{
            "x": float(d),
            "y": vid2views[id],
            "id": id,
            # "title": vid2title[id],
        } for d,id in zip(distances, done_ids)]
    }

    with open(os.path.join(RESULTS_DIR, "video_deviation", channel+".json"), "w") as f:
        json.dump(result, f)